# CS 514 Applied AI

## ASHRAE - Great Energy Prediction 3

https://www.kaggle.com/c/ashrae-energy-prediction/overview

# 1. Importing all the Libraries

In [1]:
#Ignoring all System warnings
import warnings
warnings.filterwarnings('ignore')

#Importing Garbage Collector and the system call to os
import gc
import os

#Importing all the Scientific Libraries
import pandas as pd
import numpy as np
import scipy as sc

#Importing all the plotting libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# 2. Reading the Data

In [2]:
path = "D:/KaggleProjects/ASHRAE/ashrae-energy-prediction/"

In [3]:
print(os.listdir(path))

['building_metadata.csv', 'sample_submission.csv', 'test.csv', 'train.csv', 'weather_test.csv', 'weather_train.csv']


In [9]:
train_df = pd.read_csv(path + "train.csv")

building_df = pd.read_csv(path + "building_metadata.csv")

weather_train_df = pd.read_csv(path + "weather_train.csv")

weather_test_df = pd.read_csv(path + "weather_test.csv")

test_df = pd.read_csv(path + "test.csv")

## Know more about the data

Applying df.describe() functions to every dataframe to know about the stats of the data

In [17]:
print("Describing the Train data\n",train_df.describe())
print("\n")
print("Describing the building data\n",building_df.describe())
print("\n")
print("Describing the weather train data\n",weather_train_df.describe())
print("\n")
print("Describing the weather test data\n",weather_test_df.describe())
print("\n")
print("Describing the Test data\n",test_df.describe())
print("\n")

Describing the Train data
         building_id         meter  meter_reading
count  2.021610e+07  2.021610e+07   2.021610e+07
mean   7.992780e+02  6.624412e-01   2.117121e+03
std    4.269133e+02  9.309921e-01   1.532356e+05
min    0.000000e+00  0.000000e+00   0.000000e+00
25%    3.930000e+02  0.000000e+00   1.830000e+01
50%    8.950000e+02  0.000000e+00   7.877500e+01
75%    1.179000e+03  1.000000e+00   2.679840e+02
max    1.448000e+03  3.000000e+00   2.190470e+07


Describing the building data
            site_id  building_id    square_feet   year_built  floor_count
count  1449.000000  1449.000000    1449.000000   675.000000   355.000000
mean      6.952381   724.000000   92111.776398  1967.957037     3.740845
std       5.003432   418.434583  110769.950997    31.054030     3.333683
min       0.000000     0.000000     283.000000  1900.000000     1.000000
25%       3.000000   362.000000   23012.000000  1949.000000     1.000000
50%       5.000000   724.000000   57673.000000  1970.000000   

Applying df.shape() function to every df to know the number of rows and columns in the df.


Here the result will be (x,y) where x is the number of row and y is the number of column

In [19]:
print("Shape of Train data: ",train_df.shape)
print("Shape of building data: ",building_df.shape)
print("Shape of weather train data: ",weather_train_df.shape)
print("Shape of weather test data: ",weather_test_df.shape)
print("Shape of Test data: ",test_df.shape)

Shape of Train data:  (20216100, 4)
Shape of building data:  (1449, 6)
Shape of weather train data:  (139773, 9)
Shape of weather test data:  (277243, 9)
Shape of Test data:  (41697600, 4)


Applying df.head() to display rows of the dataframe

In [22]:
train_df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [23]:
building_df.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


Here we see primary_use is a categorical variable which can be encoded using LabelEncoder().

Below we will look at all the unique categories of the primary_use column in the building_df.

In [37]:
building_df['primary_use'].unique().tolist()

[0, 4, 6, 1, 7, 11, 8, 9, 15, 2, 10, 3, 14, 13, 5, 12]

Label Encoding refers to converting the labels into numeric form so as to convert it into the machine-readable form

Example :
<br>
Suppose we have a column Height in some dataset.

Height = ["small","medium","Tall"]

Then after encoding the Height will become as: Height = [0,1,2]

In [35]:
le = LabelEncoder()
building_df['primary_use'] = le.fit_transform(building_df['primary_use'])

In [36]:
building_df.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,0,7432,2008.0,NaN
1,0,1,0,2720,2004.0,NaN
2,0,2,0,5376,1991.0,NaN
3,0,3,0,23685,2002.0,NaN
4,0,4,0,116607,1975.0,NaN


In [24]:
weather_train_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [25]:
weather_test_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2017-01-01 00:00:00,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1
2,0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1
3,0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1
4,0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6


In [26]:
test_df.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00
2,2,2,0,2017-01-01 00:00:00
3,3,3,0,2017-01-01 00:00:00
4,4,4,0,2017-01-01 00:00:00


# 3. Reducing the Memory Usage

Some features take up more memory space than they should, and since there is too much data, this is critical step to reduce the memory use by the data.

In [42]:
# we can count the actual memory usage using the following command
print("Memory used by train_df:")
train_df.info(memory_usage='deep')
print("\n Memory used by building_df:")
building_df.info(memory_usage='deep')
print("\n Memory used by weather_train_df:")
weather_train_df.info(memory_usage='deep')
print("\n Memory used by weather_test_df:")
weather_test_df.info(memory_usage='deep')
print("\n Memory used by test_df:")
test_df.info(memory_usage='deep')

Memory used by train_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
building_id      int64
meter            int64
timestamp        object
meter_reading    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.9 GB

 Memory used by building_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
site_id        1449 non-null int64
building_id    1449 non-null int64
primary_use    1449 non-null int32
square_feet    1449 non-null int64
year_built     675 non-null float64
floor_count    355 non-null float64
dtypes: float64(2), int32(1), int64(3)
memory usage: 62.3 KB

 Memory used by weather_train_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
site_id               139773 non-null int64
timestamp             139773 non-null object
air_temperature       139718 non-null float64
cloud_coverage  

In [51]:
# we can check how much space each column is actually taking
# the numbers are in bytes, not kilobytes
train_df.memory_usage('deep')

Index                   80
building_id      161728800
meter            161728800
timestamp        161728800
meter_reading    161728800
dtype: int64

![](D:\KaggleProjects\ASHRAE\ashrae-energy-prediction\table.png)